## Time-based manual segmentation


In [45]:
import os
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import dark

midi_file = os.path.join("data", "all-time", "20240117-64-2-t_72-80.mid")

MIDIPlayer(midi_file, 400, styler=dark)

In [37]:
import numpy as np
from pathlib import Path

def divide_midi(input_file, output_folder, segment_length=8):
    filename = Path(input_file).stem
    midi_data = pretty_midi.PrettyMIDI(input_file)
    total_length = midi_data.get_end_time()

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for start in range(0, int(total_length), segment_length):
        end = start + segment_length
        segment_midi = pretty_midi.PrettyMIDI()
        instrument = pretty_midi.Instrument(program=midi_data.instruments[0].program, name=f"{filename}_{start}-{end}")

        # add notes from the original MIDI that fall within the current segment
        for note in midi_data.instruments[0].notes:
            if start <= note.start < end:
                new_note = pretty_midi.Note(
                    velocity=note.velocity,
                    pitch=note.pitch,
                    start=note.start - start,
                    end=min(note.end, end) - start
                )
                instrument.notes.append(new_note)

        # write out
        segment_midi.instruments.append(instrument)
        segment_midi.write(os.path.join(output_folder, f'{filename}_{start}-{end}.mid'))

In [43]:
midi_folder = os.path.join("data", "trimmed outputs")

for filename in os.listdir(midi_folder):
  # dir_out = os.path.join("data", "time", f"{Path(filename).stem}")
  dir_out = os.path.join("data", "all-time")
  print(f"Segmenting {filename} to {dir_out}")
  divide_midi(os.path.join(midi_folder, filename), dir_out)

Segmenting 20240121-66-2-t.mid to data/inputs/all-time
Segmenting 20231228-72-1-t.mid to data/inputs/all-time
Segmenting 20240121-70-9-t.mid to data/inputs/all-time
Segmenting 20240121-66-3-t.mid to data/inputs/all-time
Segmenting 20240121-66-1-t.mid to data/inputs/all-time
Segmenting 20240121-70-8-t.mid to data/inputs/all-time
Segmenting 20240121-70-6-t.mid to data/inputs/all-time
Segmenting 20240117-64-3-t.mid to data/inputs/all-time
Segmenting 20240121-70-4-t.mid to data/inputs/all-time
Segmenting 20240117-64-1-t.mid to data/inputs/all-time
Segmenting 20231220-80-5-t.mid to data/inputs/all-time
Segmenting 20231227-80-2-t.mid to data/inputs/all-time
Segmenting 20231227-80-6-t.mid to data/inputs/all-time
Segmenting 20240117-64-5-t.mid to data/inputs/all-time
Segmenting 20231220-80-1-t.mid to data/inputs/all-time
Segmenting 20231220-80-3-t.mid to data/inputs/all-time
Segmenting 20240121-70-2-t.mid to data/inputs/all-time
Segmenting 20231227-80-4-t.mid to data/inputs/all-time
Segmenting